In [1]:
"""
Mask R-CNN
Configurations and data loading code for MS COCO.

Copyright (c) 2017 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla

------------------------------------------------------------

Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:

    # Train a new model starting from pre-trained COCO weights
    python3 coco.py train --dataset=/path/to/coco/ --model=coco

    # Train a new model starting from ImageNet weights
    python3 coco.py train --dataset=/path/to/coco/ --model=imagenet

    # Continue training a model that you had trained earlier
    python3 coco.py train --dataset=/path/to/coco/ --model=/path/to/weights.h5

    # Continue training the last model you trained
    python3 coco.py train --dataset=/path/to/coco/ --model=last

    # Run COCO evaluatoin on the last model you trained
    python3 coco.py evaluate --dataset=/path/to/coco/ --model=last
"""

import os
import time
import numpy as np
%load_ext autoreload
%autoreload 2
print(' breakpoint 1')
# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.
# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

from   mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.dataset as dataset

# Root directory of the project
ROOT_DIR = os.getcwd()

# Path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")



 breakpoint 1


D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from coco import CocoDataset, CocoConfig, evaluate_coco, build_coco_results
import pprint as pp
pp = pp.PrettyPrinter(indent=4)
import argparse

# Parse command line arguments
parser = argparse.ArgumentParser(description='Train Mask R-CNN on MS COCO.')
parser.add_argument("command",
                    metavar="<command>",
                    help="'train' or 'evaluate' on MS COCO")
parser.add_argument('--dataset', required=True,
                    metavar="/path/to/coco/",
                    help='Directory of the MS-COCO dataset')
parser.add_argument('--model', required=True,
                    metavar="/path/to/weights.h5",
                    help="Path to weights .h5 file or 'coco'")
parser.add_argument('--logs', required=False,
                    default=DEFAULT_LOGS_DIR,
                    metavar="/path/to/logs/",
                    help='Logs and checkpoints directory (default=logs/)')
parser.add_argument('--limit', required=False,
                    default=500,
                    metavar="<image count>",
                    help='Images to use for evaluation (defaults=500)')
# args = parser.parse_args()
args = parser.parse_args("train --dataset E:\MLDatasets\coco2014 --model mask_rcnn_coco.h5 --limit 10".split())
# pp.pprint(options)
pp.pprint(args)
print("Model:   ", args.model)
print("Dataset: ", args.dataset)
print("Logs:    ", args.logs)
print("Limit:   ", args.limit)

# Configurations

 breakpoint 1
 breakpoint 2
Namespace(command='train', dataset='E:\\MLDatasets\\coco2014', limit='10', logs='E:\\git_projs\\Mask_RCNN_2\\logs', model='mask_rcnn_coco.h5')
Model:    mask_rcnn_coco.h5
Dataset:  E:\MLDatasets\coco2014
Logs:     E:\git_projs\Mask_RCNN_2\logs
Limit:    10


In [3]:
# Configurations
if args.command == "train":
    config = CocoConfig()
else:
    class InferenceConfig(CocoConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
        DETECTION_MIN_CONFIDENCE = 0
    config = InferenceConfig()
config.display()

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
# tf_config = tf.ConfigProto()
# tf_config.gpu_options.per_process_gpu_memory_fraction = 0.55
# set_session(tf.Session(config=tf_config))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.55)
set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING     

In [148]:
# Create model
if args.command == "train":
    print('setup model for Training ')
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)
else:
    print('setup model for Inference ')
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=args.logs)

setup model for Training 
 IMAGE SHAPE is : 1024    1024
<class 'list'>
Tensor("rpn_class_logits_13/concat:0", shape=(?, ?, 2), dtype=float32) rpn_class_logits_13/concat:0
Tensor("rpn_class_13/concat:0", shape=(?, ?, 2), dtype=float32) rpn_class_13/concat:0
Tensor("rpn_bbox_13/concat:0", shape=(?, ?, 4), dtype=float32) rpn_bbox_13/concat:0
>>> MaskRCNN build complete
>>> MaskRCNN initialization complete


In [135]:
# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
layers    = [layer for layer in mdl.layers]          # all layer outputs
len(layers)

236

In [146]:
for i in range(len(layers)):
    lay = layers[i]
    print('** Layer {} , ({})'.format(i, lay.name))
    print('===========================================')
    for innode in lay._inbound_nodes:
        print('\t Inbound Node:  ', innode)
        print('\t               From Layers: ', [inlayer.name for inlayer in innode.inbound_layers])
        print('\t                    Shapes: ', [inshape for inshape in innode.input_shapes])
        print('\t               To   Layers: ', innode.outbound_layer.name)
#         pp.pprint(innode.__dict__)
#         print('\t                    Shapes: ', [outshape for outshape in innode.output_shapes])        
    print('\t-------------------------------------------')
    print('\t Layer:', lay.name, '\t\t type  ' , lay)
#     print('-------------------------------------------')
#     pp.pprint(lay.__dict__)
#     print('-------------------------------------------')
    print('\t     Layer Output Shapes: ', [outshape for outshape in innode.output_shapes])        
    print('\t-------------------------------------------')
    for outnode in lay._outbound_nodes:
        print('\t Outbound Node: ', outnode)
#         print('\t               From Layers: ', [inlayer.name for inlayer in outnode.inbound_layers])
#         print('\t                    Shapes: ', [inshape for inshape in outnode.input_shapes])
        print('\t               To   Layer: ', outnode.outbound_layer.name)
#         print('                    Shapes: ', [outshape for outshape in outnode.output_shapes])    
    
#         pp.pprint(outnode.__dict__)
    print('===========================================')

** Layer 0 , (input_image)
	 Inbound Node:   <keras.engine.topology.Node object at 0x000001A612611CC0>
	               From Layers:  []
	                    Shapes:  [(None, 1024, 1024, 3)]
	               To   Layers:  input_image
	-------------------------------------------
	 Layer: input_image 		 type   <keras.engine.topology.InputLayer object at 0x000001A612611C18>
	     Layer Output Shapes:  [(None, 1024, 1024, 3)]
	-------------------------------------------
	 Outbound Node:  <keras.engine.topology.Node object at 0x000001A6125BAE10>
	               To   Layer:  zero_padding2d_6
** Layer 1 , (zero_padding2d_6)
	 Inbound Node:   <keras.engine.topology.Node object at 0x000001A6125BAE10>
	               From Layers:  ['input_image']
	                    Shapes:  [(None, 1024, 1024, 3)]
	               To   Layers:  zero_padding2d_6
	-------------------------------------------
	 Layer: zero_padding2d_6 		 type   <keras.layers.convolutional.ZeroPadding2D object at 0x000001A61E0F4D68>
	

In [83]:
model.keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, 1024, 1024, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 1030, 1030, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 512, 512, 64) 9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, 512, 512, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
for idx,layer in enumerate(layers):
    print('>layer name : {}  '.format(layer.name))
    print('   layer {}  type: {}'.format(idx,layer.name,layer))
#     pp.pprint(layer._inbound_nodes[0].__dict__)
    print("   >> input_shape {} inbound node {} ".format(layer._inbound_nodes[0].input_shapes,layer._inbound_nodes[0]))
#     print('>> inbound node output_shape {}'.format(layer._inbound_nodes[0].output_shapes))
#     if isinstance("layer", mrcnn.model.BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)

#     pp.pprint(layer.__dict__)
    print("   >> outbound shape{} input_shape {}".format(layer._outbound_nodes[0].input_shapes,layer._outbound_nodes[0]))
#     print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    print('-------------------------------------------')
    #layer.name, layer.input._keras_shape, layer.output._keras_shape

In [127]:
def generate_anchors(scales, ratios, shape, feature_stride, anchor_stride):
    """
    scales: 1D array of anchor sizes in pixels. Example: [32, 64, 128]
    ratios: 1D array of anchor ratios of width/height. Example: [0.5, 1, 2]
    shape: [height, width] spatial shape of the feature map over which
            to generate anchors.
    feature_stride: Stride of the feature map relative to the image in pixels.
    anchor_stride: Stride of anchors on the feature map. For example, if the
        value is 2 then generate anchors for every other feature map pixel.
    """
    print('Generate Anchors: \n ================== \n scales: ',scales, 'Anchor Ratios:', ratios, 'Anchor stride', anchor_stride,'Feat. Shape:', shape, ' Feat stride', feature_stride)
    # Get all combinations of scales and ratios
    scales, ratios = np.meshgrid(np.array(scales), np.array(ratios))
    scales = scales.flatten()
    ratios = ratios.flatten()
    print('Scales: ',scales,'\n Ratios:', ratios)
    # Enumerate heights and widths from scales and ratios
    heights = scales / np.sqrt(ratios)
    widths = scales * np.sqrt(ratios)

    print(' heights: ',heights,'\n widths: ', widths)
    
    # Enumerate shifts in feature space
    shifts_y = np.arange(0, shape[0], anchor_stride) * feature_stride
    shifts_x = np.arange(0, shape[1], anchor_stride) * feature_stride
#     print(' shifts_x : ',shifts_x.shape, '\n shifts_y : ',  shifts_y.shape )    
    shifts_x, shifts_y = np.meshgrid(shifts_x, shifts_y)
#     print(' shifts_x : ',shifts_x.shape,shifts_x,'\n shifts_y : ',  shifts_y.shape,shifts_y)
    
    # Enumerate combinations of shifts, widths, and heights
    box_widths, box_centers_x = np.meshgrid(widths, shifts_x)
    box_heights, box_centers_y = np.meshgrid(heights, shifts_y)
#     print('box_widths : ',box_widths.shape,'\n',box_widths,'\n box_centers_x: ',  box_centers_x.shape,'\n',box_centers_x)
#     print('box_heights : ',box_heights.shape,'\n',box_heights,'\n box_centers_y: ',  box_centers_y.shape,'\n',box_centers_y)

    # Reshape to get a list of (y, x) and a list of (h, w)
    # This will create a array of (y,x) which has 
    box_centers = np.stack([box_centers_y, box_centers_x], axis=2)
#     print('box_centers : ', box_centers.shape,'\n',box_centers)
    box_centers = box_centers.reshape([-1, 2])
    box_sizes   = np.stack([box_heights, box_widths], axis=2).reshape([-1, 2])
#     print('box_centers : ', box_centers.shape,'\n',box_centers)
#     print('box_sizes   : ', box_sizes.shape,'\n',box_sizes)

    # Convert to corner coordinates (y1, x1, y2, x2)
    boxes = np.concatenate([box_centers - 0.5 * box_sizes, box_centers + 0.5 * box_sizes], axis=1)
#     print('boxes   : ', boxes.shape,'\n',boxes)

    return boxes

In [128]:
# self.anchors = utils.generate_pyramid_anchors(config.RPN_ANCHOR_SCALES,
#                                               config.RPN_ANCHOR_RATIOS,
#                                               config.BACKBONE_SHAPES,
#                                               config.BACKBONE_STRIDES,
#                                               config.RPN_ANCHOR_STRIDE)
# Anchors
# [anchor_count, (y1, x1, y2, x2)]
scales = config.RPN_ANCHOR_SCALES
ratios = config.RPN_ANCHOR_RATIOS
anchor_stride   = config.RPN_ANCHOR_STRIDE
feature_shapes  = config.BACKBONE_SHAPES
feature_strides = config.BACKBONE_STRIDES

print ('Anchoo scales: ',scales, 'Anchor Ratios:', ratios, 'Anchor stride', anchor_stride)
print(feature_shapes, '\n Feature strides', feature_strides)
anchors = []
for i in range(len(scales)):    
    anchors.append(generate_anchors(scales[i], ratios, feature_shapes[i],
                                    feature_strides[i], anchor_stride))
    
# return np.concatenate(anchors, axis=0)

Anchoo scales:  (32, 64, 128, 256, 512) Anchor Ratios: [0.5, 1, 2] Anchor stride 1
[[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]] 
 Feature strides [4, 8, 16, 32, 64]
Generate Anchors: 
 scales:  32 Anchor Ratios: [0.5, 1, 2] Anchor stride 1 Feat. Shape: [256 256]  Feat stride 4
Scales:  [32 32 32] 
 Ratios: [0.5 1.  2. ]
 heights:  [45.254834 32.       22.627417] 
 widths:  [22.627417 32.       45.254834]
Generate Anchors: 
 scales:  64 Anchor Ratios: [0.5, 1, 2] Anchor stride 1 Feat. Shape: [128 128]  Feat stride 8
Scales:  [64 64 64] 
 Ratios: [0.5 1.  2. ]
 heights:  [90.50966799 64.         45.254834  ] 
 widths:  [45.254834   64.         90.50966799]
Generate Anchors: 
 scales:  128 Anchor Ratios: [0.5, 1, 2] Anchor stride 1 Feat. Shape: [64 64]  Feat stride 16
Scales:  [128 128 128] 
 Ratios: [0.5 1.  2. ]
 heights:  [181.01933598 128.          90.50966799] 
 widths:  [ 90.50966799 128.         181.01933598]
Generate Anchors: 
 scales:  256 Anchor Ratios: [0.5, 1, 2] An

In [123]:
for i in anchors :
    print(i.shape)
np.concatenate(anchors, axis=0).shape

(196608, 4)
(49152, 4)
(12288, 4)
(3072, 4)
(768, 4)


(261888, 4)